In [1]:
import asyncio
import ascon

async def handle_emitter(key_init_receiver, nonce_init_receiver, reader, writer):
        
    # Lê os dados associados
    data = await reader.readuntil(b'_')
    associateddata = data.strip()
    size = int(associateddata.decode()[:-1])
    
    #Lê a mensagem
    data = await reader.readexactly(size)
    ciphertext = data.strip()
    
    #gerar chave e nonce
    key = ascon.hash(key_init_receiver.encode(), variant="Ascon-Xof", hashlength=16)
    nonce = ascon.hash(nonce_init_receiver.encode(), variant="Ascon-Xof", hashlength=16)
    
    #decifrar a mensagem
    plaintext = ascon.decrypt(key, nonce, associateddata, ciphertext, variant="Ascon-128")
    
    if plaintext is None:
        print("Autentication failed. Message compromised.")
        writer.close()
    else:
        print(f"Message: {plaintext.decode()}")
        writer.close()
    
#receiver
async def receiver(key_init_receiver, nonce_init_receiver):
    server = await asyncio.start_server(
        lambda reader, writer: handle_emitter(
            key_init_receiver, nonce_init_receiver, reader, writer), '127.0.0.1', 8888)

    print("Receiver ready...\n")

    async with server:
        await server.serve_forever()


#Emitter
async def send_message( key_init_emitter, nonce_init_emitter):
    
    #establecer concexão
    reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

    plaintext = input("Enter message: ").encode()
    #associateddata = input("Enter associated data: ")
    
    #dados associados = comprimento do text
    associateddata = (str(len(plaintext)+16) + '_').encode()

    #gerar chaves aliatórias
    key = ascon.hash(key_init_emitter.encode(), variant="Ascon-Xof", hashlength=16)
    nonce = ascon.hash(nonce_init_emitter.encode(), variant="Ascon-Xof", hashlength=16)
    
    #cifrar texto
    ciphertext = ascon.encrypt(
        key, nonce, associateddata, plaintext, variant="Ascon-128")
    
    #anexar dados associados
    message = associateddata + ciphertext

    # Envia a mensagem para o servidor
    writer.write(message)
    await writer.drain()

    # Fecha a conexão
    writer.close()
    await writer.wait_closed()
    


In [2]:
#Receiver
key_init_receive = "27082001"
nonce_init_receive = "12071972"

asyncio.create_task(receiver(key_init_receive, nonce_init_receive))  # Inicia o servidor

<Task pending name='Task-5' coro=<receiver() running at C:\Users\Utilizador\AppData\Local\Temp\ipykernel_11700\3947970274.py:40>>

Receiver ready...



In [3]:
#Emitter, caso certo

key_init_emitter = "27082001"
nonce_init_emitter = "12071972"

async def emitter():
        
    await send_message

# Envia a mensagem
await send_message( key_init_emitter, nonce_init_emitter)


Message: Olá, teste 0101010101010101010101010101010101010101010101010101010101010101010101 teste
AD: De Lucas



In [4]:
#Emitter, key errada

key_init_emitter = "27082002"
nonce_init_emitter = "12071972"

# Envia a mensagem
await send_message(key_init_emitter, nonce_init_emitter) 


Autentication failed. Message compromised.


In [5]:
#Emitter, nonce errado

key_init_emitter = "270801"
nonce_init_emitter = "12071973"

# Envia a mensagem
await send_message(key_init_emitter, nonce_init_emitter) 


Autentication failed. Message compromised.
